In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from tqdm import tqdm
import os

from core.robot import Robot
from core.dataloader import EncoderIMULoader, LidarLoader
from core.map import Map
from core.load_data import load_data
import matplotlib.pyplot as plt

In [3]:

experiments = {
    "N": [10, 100, 1000],
    "lidar": [True],
    "sigma": [0, 0.01, 0.05]
}


# experiments = {
#     "N": [100],
#     "lidar": [True],
#     "sigma": [0]
# }

os.makedirs("results", exist_ok=True)
for N in experiments["N"]:
    for lidar in experiments["lidar"]:
        for sigma in experiments["sigma"]:
            print(f"N: {N}, lidar: {lidar}, sigma: {sigma}")

            save_dir = f"results/N_{N}_lidar_{lidar}_sigma_{sigma}"
            os.makedirs(save_dir, exist_ok=True)

            dataset_num = 20
            dataset_dir = "data"
            data = load_data(dataset_dir, dataset_num)

            robot = Robot(N = N, sigma = sigma, parallize=False, dataset_dir=save_dir)

            encoder = EncoderIMULoader(data)
            lidar_data = LidarLoader(data)

            encoder_stamps = encoder.encoder_stamps
            lidar_stamps = lidar_data.lidar_stamps

            timestamps = np.concatenate((encoder_stamps, lidar_stamps))
            timestamps.sort()

            for index, timestamp in tqdm(enumerate(timestamps)):
                if timestamp in encoder_stamps:
                    encoder_index = np.where(encoder_stamps == timestamp)[0][0]

                    if encoder_index == 0:
                        continue

                    motion_model = encoder.motion_model(encoder_index)
                    robot.predict(motion_model)

                elif timestamp in lidar_stamps:
                    lidar_index = np.where(lidar_stamps == timestamp)[0][0]
                    lidar_points = lidar_data(lidar_index)

                    robot.update(lidar_points.points)
                else:
                    print("Something went wrong")

            #plot trajectory
            robot.plot_trajectory()

            # plot map
            robot.map.plot()

            import pickle
            with open(f"{save_dir}/robot.pickle", "wb") as f:
                pickle.dump(robot, f)

            with open(f"{save_dir}/map.pickle", "wb") as f:
                pickle.dump(robot.map, f)

            del robot

N: 100, lidar: True, sigma: 0


399it [01:04,  5.92it/s]

In [5]:
experiments = {
    "N": [1],
    "lidar": [False],
    "sigma": [0, 0.001, 0.01, 0.05, 0.1, 0.5]
}

os.makedirs("results", exist_ok=True)
for N in experiments["N"]:
    for lidar in experiments["lidar"]:
        for sigma in experiments["sigma"]:
            print(f"N: {N}, lidar: {lidar}, sigma: {sigma}")

            save_dir = f"results/N_{N}_lidar_{lidar}_sigma_{sigma}"
            os.makedirs(save_dir, exist_ok=True)

            dataset_num = 20
            dataset_dir = "data"
            data = load_data(dataset_dir, dataset_num)

            robot = Robot(N = N, sigma = sigma, parallize=False, dataset_dir=save_dir)

            encoder = EncoderIMULoader(data)
            lidar_data = LidarLoader(data)

            encoder_stamps = encoder.encoder_stamps
            lidar_stamps = lidar_data.lidar_stamps

            timestamps = np.concatenate((encoder_stamps, lidar_stamps))
            timestamps.sort()

            for index, timestamp in tqdm(enumerate(timestamps)):
                if timestamp in encoder_stamps:
                    encoder_index = np.where(encoder_stamps == timestamp)[0][0]

                    if encoder_index == 0:
                        continue

                    motion_model = encoder.motion_model(encoder_index)
                    robot.predict(motion_model)

                elif timestamp in lidar_stamps:
                    continue
                    lidar_index = np.where(lidar_stamps == timestamp)[0][0]
                    lidar_points = lidar_data(lidar_index)

                    robot.update(lidar_points.points)
                else:
                    print("Something went wrong")

            particle = robot.particles[0]
            particle_history = particle.history
            plt.plot(particle_history[:, 0], particle_history[:, 1])
            plt.savefig(f"{save_dir}/particle_history.png")
            plt.close()

            import pickle
            with open(f"{save_dir}/robot.pickle", "wb") as f:
                pickle.dump(robot, f)

            with open(f"{save_dir}/map.pickle", "wb") as f:
                pickle.dump(robot.map, f)

            del robot

N: 1, lidar: False, sigma: 0


9918it [00:00, 44633.66it/s]


N: 1, lidar: False, sigma: 0.001


9918it [00:00, 44946.11it/s]


N: 1, lidar: False, sigma: 0.01


...

9918it [00:00, 42812.54it/s]


N: 1, lidar: False, sigma: 0.1


9918it [00:00, 44677.67it/s]


N: 1, lidar: False, sigma: 0.5


9918it [00:00, 44396.34it/s]


In [3]:
experiments = {
    "N": [1],
    "lidar": [True],
    "encoder": [False],
    "sigma": [0]
}

os.makedirs("results", exist_ok=True)
for N in experiments["N"]:
    for lidar in experiments["lidar"]:
        for sigma in experiments["sigma"]:
            for encoder in experiments["encoder"]:
                print(f"N: {N}, lidar: {lidar}, sigma: {sigma}, encoder: {encoder}")

                save_dir = f"results/N_{N}_lidar_{lidar}_sigma_{sigma}_encoder_{encoder}"
                os.makedirs(save_dir, exist_ok=True)

                dataset_num = 20
                dataset_dir = "data"
                data = load_data(dataset_dir, dataset_num)

                robot = Robot(N = N, sigma = sigma, parallize=False, dataset_dir=save_dir)

                encoder = EncoderIMULoader(data)
                lidar_data = LidarLoader(data)

                encoder_stamps = encoder.encoder_stamps
                lidar_stamps = lidar_data.lidar_stamps

                timestamps = np.concatenate((encoder_stamps, lidar_stamps))
                timestamps.sort()

                for index, timestamp in tqdm(enumerate(timestamps)):
                    if timestamp in encoder_stamps:
                        encoder_index = np.where(encoder_stamps == timestamp)[0][0]

                        if encoder_index == 0:
                            continue

                        motion_model = encoder.motion_model(encoder_index)
                        robot.predict(motion_model)

                    elif timestamp in lidar_stamps:
                        lidar_index = np.where(lidar_stamps == timestamp)[0][0]
                        lidar_points = lidar_data(lidar_index)

                        robot.update(lidar_points.points)
                        break
                    else:
                        print("Something went wrong")

                #plot trajectory
                robot.plot_trajectory()

                # plot map
                robot.map.plot()

                import pickle
                with open(f"{save_dir}/robot.pickle", "wb") as f:
                    pickle.dump(robot, f)

                with open(f"{save_dir}/map.pickle", "wb") as f:
                    pickle.dump(robot.map, f)

                del robot

N: 1, lidar: True, sigma: 0, encoder: False


1it [00:00,  8.64it/s]
